In [17]:
import warnings
from idlelib.pyparse import trans

from pyexpat import features

from torch.onnx.symbolic_opset9 import permute

warnings.filterwarnings("ignore")

## 1.1 Data preprocessing

In [18]:
# necessary imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from tqdm.notebook import tqdm

from torch.utils.tensorboard import SummaryWriter

import torchvision.transforms as transforms
from torchvision.io import read_image

In [19]:
from torchvision.transforms import v2


def load_img(fname):
    """
    Load an image from file, do transformation (including possible augmentation) and return it as torch.tensor

    :param fname: path to jpg image
    """
    img = read_image(fname)
    x = img / 255.

    # Write your code here
    transform = transforms.Compose([
        transforms.Resize((224, 224)),  # Resize to a consistent input size
        transforms.RandomHorizontalFlip(p=0.5),  # 50% chance of flipping
        transforms.ColorJitter(brightness=0.1,  # Slight color adjustments
                               contrast=0.1,
                               saturation=0.1,
                               hue=0.05),
        transforms.RandomRotation(degrees=5),  # Small rotation to keep natural orientation
        transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet normalization
                             std=[0.229, 0.224, 0.225]),
        transforms.ConvertImageDtype(torch.float32)  # Convert to float32 for model compatibility
    ])

    return transform(x)

In [20]:
from os import walk

decode = {
    0: "1-5", 1: "5-10", 2: "10-15", 3: "15-20", 4: "20-25", 5: "25-30", 6: "30-40", 7: "40-50", 8: "50-60", 9: "60-70",
    10: "70+"
}
encode = {
    "1-5": 0, "5-10": 1, "10-15": 2, "15-20": 3, "20-25": 4, "25-30": 5, "30-40": 6, "40-50": 7, "50-60": 8, "60-70": 9,
    "70+": 10
}
img_path = "facial-age"
# _id = []
# _age = []
# dirs_ages = []
# for (dirpath, dirnames, filenames) in walk(img_path):
#     dirs_ages.extend(dirnames)
#     break
# for dire in dirs_ages:
#     for (dirpath, dirnames, filenames) in walk(img_path + "/" + dire):
#         _id.extend(list(map(lambda x:dire+"/"+x,filenames)))
#         if 1<=int(dire)<5:
#             _age.extend(["1-5" for _ in range(len(filenames))])
#         elif 5<=int(dire)<10:
#             _age.extend(["5-10" for _ in range(len(filenames))])
#         elif 10<=int(dire)<15:
#             _age.extend(["10-15" for _ in range(len(filenames))])
#         elif 15<=int(dire)<20:
#             _age.extend(["15-20" for _ in range(len(filenames))])
#         elif 20<=int(dire)<25:
#             _age.extend(["20-25" for _ in range(len(filenames))])
#         elif 25<=int(dire)<30:
#             _age.extend(["25-30" for _ in range(len(filenames))])
#         elif 30<=int(dire)<40:
#             _age.extend(["30-40" for _ in range(len(filenames))])
#         elif 40<=int(dire)<50:
#             _age.extend(["40-50" for _ in range(len(filenames))])
#         elif 50<=int(dire)<60:
#             _age.extend(["50-60" for _ in range(len(filenames))])
#         elif 60<=int(dire)<70:
#             _age.extend(["60-70" for _ in range(len(filenames))])
#         elif 70<=int(dire):
#             _age.extend(["70+" for _ in range(len(filenames))])
#         break
# 
# features_data = {"ID": _id, "Age": _age}
# features = pd.DataFrame(features_data)
# features.to_csv("data.csv", index=False)
features = pd.read_csv("data.csv")
features['Age'] = list(map(lambda x: encode[x], features['Age']))


In [21]:
#facial-age

In [22]:
#img_path = "/kaggle/input/pmldl-week-2-dnn-training-with-tracking-tools/archive"

# Image attributes
#train_features = pd.read_csv(f"{img_path}/train.csv")

# Load and transform images 
images = torch.stack(
    [load_img(f"{img_path}/{item['ID']}") for _, item in features.iterrows()])

# Write your code here
# Select label(s) from train_features
labels = features.get('Age')
# Leave values that only 1 or 0 and convert to float just for simplicity
labels = torch.from_numpy(labels.to_numpy()).float()

In [23]:
# Just some checking of shapes
images.shape, labels.shape

(torch.Size([9773, 3, 224, 224]), torch.Size([9773]))

## 1.3 Data loaders creation

In [24]:
from torch.utils.data import TensorDataset, DataLoader

processed_dataset = TensorDataset(images, labels)

# Write your code here
# Set proportion and split dataset into train and validation parts
proportion = 0.9

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    processed_dataset,
    [int(len(images) * 0.7) + 1, int(len(images) * 0.15) + 1, int(len(images) * 0.15)],
)


In [25]:
# Create Dataloaders for training and validation 
# Dataloader is iterable object over dataset
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)


## 2. Training


In [26]:
import torch
import torch.nn as nn
import torch.nn.functional as F


# class CNNClassificationModel(nn.Module):
#     """
#     MLP (multi-layer perceptron) based classification model for MNIST
#     """
# 
#     def __init__(self, num_classes=20):
#         super(CNNClassificationModel, self).__init__()
# 
#         # Add fully connected layers to nn.Sequential to create MLP
#         # First layer should take 28x28 vector
#         # last layer should return vector of size num_classes
#         # do not forget to add activation function between layers
# 
#         self.block1 = nn.Sequential(
#             nn.BatchNorm2d(3),
#             nn.Conv2d(3, 4, kernel_size=(3, 3), stride=2),
#             nn.ReLU(),
#             nn.BatchNorm2d(4),
#         )
# 
#         self.block2 = nn.Sequential(
#             nn.Conv2d(4, 8, kernel_size=(3, 3)),
#             nn.ReLU(),
#             nn.BatchNorm2d(8),
#         )
# 
#         self.out = nn.Sequential(
#             nn.Linear(95048, 1024),
#             nn.ReLU(),
#             nn.Dropout(0.3),
#             nn.Linear(1024, 32),
#             nn.ReLU(),
#             nn.Dropout(0.4),
#             nn.Linear(32, num_classes),
#         )
# 
#     def forward(self, x):
#         x = self.block1(x)
#         x = self.block2(x)
#         x = x.view(x.size(0), -1)
#         x = self.out(x)
#         return x




In [27]:

class CNNClassificationModel(nn.Module):
    """
    CNN-based classification model for age prediction with 20 classes.
    """

    def __init__(self, num_classes=11):  # Changed to 20 classes
        super(CNNClassificationModel, self).__init__()

        self.block1 = nn.Sequential(
            nn.BatchNorm2d(3),
            nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.BatchNorm2d(32)
        )

        self.block2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.BatchNorm2d(64)
        )

        self.block3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.BatchNorm2d(128)
        )

        self.block4 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.BatchNorm2d(256)
        )

        # Global Average Pooling
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))

        # Fully connected layers with reduced complexity
        self.fc = nn.Sequential(
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.4),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, num_classes)
        )

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)

        x = self.global_avg_pool(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc(x)
        return x


In [28]:
def train(
        model,
        optimizer,
        loss_fn,
        train_loader,
        val_loader,
        writer=None,
        epochs=1,
        device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
        ckpt_path="best.pt",
):
    # best score for checkpointing
    best = 0.51

    # iterating over epochs
    for epoch in range(epochs):
        # training loop description
        train_loop = tqdm(
            enumerate(train_loader, 0), total=len(train_loader), desc=f"Epoch {epoch}"
        )
        model.train()
        train_loss = 0.0
        # iterate over dataset 
        for data in train_loop:
            # Write your code here
            # Move data to a device, do forward pass and loss calculation, do backward pass and run optimizer
            id, (inputs, labels) = data
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            labels = labels.type(torch.int64)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_loop.set_postfix({"loss": loss.item()})
        # write loss to tensorboard
        if writer:
            writer.add_scalar("Loss/train", train_loss / len(train_loader), epoch)

        # Validation
        correct = 0
        total = 0
        with torch.no_grad():
            model.eval()  # evaluation mode
            val_loop = tqdm(enumerate(val_loader, 0), total=len(val_loader), desc="Val")
            for data in val_loop:
                id, (inputs, labels) = data

                # Write your code here
                # Get predictions and compare them with labels
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = model(inputs)
                labels = labels.type(torch.int64)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)

                for i, j in zip(predicted, labels):
                    if i == j: correct += 1

                val_loop.set_postfix({"acc": correct / total})

            if correct / total > best:
                torch.save(model.state_dict(), ckpt_path)
                best = correct / total


## 2.3 Combining everything together

In [29]:
import torch.optim as optim

# Write your code here
# Pick optimizer from torch.optim and loss function loss_fn from torch.nn that suits best the model
# SummaryWriter is used by tensorboard and could be set None
model = CNNClassificationModel()

train(
    model=model,
    optimizer=optim.Adam(model.parameters(), lr=0.001),
    loss_fn=nn.CrossEntropyLoss(),
    train_loader=train_loader,
    val_loader=val_loader,
    device='cpu',
    writer=SummaryWriter(),
    epochs=30
)


Epoch 0:   0%|          | 0/107 [00:00<?, ?it/s]

KeyboardInterrupt: 

## 2.4 Inference
Here you need to perform inference of trained model on test data. 

Load the best checkpoint from training to the model and run inference

In [30]:
# load best checkpoint to model
model = CNNClassificationModel()
ckpt = torch.load("best.pt")
model.load_state_dict(ckpt)

<All keys matched successfully>

In [31]:
import PIL.Image


def predict(model, test_loader, device):
    """
    Run model inference on test data
    """
    predictions = []
    with torch.no_grad():
        model.eval()  # evaluation mode
        test_loop = tqdm(enumerate(test_loader, 0), total=len(test_loader), desc="Test")

        for inputs in test_loop:
            # Write your code here
            # Similar to validation part in training cell
            id, pred = inputs
            pred = pred.to(device)
            _, predicted = torch.max(model(pred).data, 1)

            # Extend overall predictions by prediction for a batch
            predictions.extend([i.item() for i in predicted])
        return predictions


def load_training_img(fname):
    rgba_image = PIL.Image.open(fname)
    rgb_image = rgba_image.convert('RGB')
    img = rgb_image

    # Write your code here
    transform = transforms.Compose([
        transforms.PILToTensor(),
        transforms.Lambda(lambda x: x/255),
        transforms.ConvertImageDtype(torch.float32),
        transforms.Resize((224, 224)),  # Resize to a consistent input size
        transforms.RandomHorizontalFlip(p=0.5),  # 50% chance of flipping
        transforms.ColorJitter(brightness=0.1,  # Slight color adjustments
                               contrast=0.1,
                               saturation=0.1,
                               hue=0.05),
        transforms.RandomRotation(degrees=5),  # Small rotation to keep natural orientation
        transforms.Normalize(mean=[0.485, 0.456, 0.406],  # ImageNet normalization
                             std=[0.229, 0.224, 0.225]),
        transforms.ConvertImageDtype(torch.float32)
    ])  # Convert to float32 for model compatibility
    transforms.ToPILImage()(transform(img)).show()

    return transform(img)

In [54]:

# process test data and run inference on it
images = torch.stack(
    [load_training_img("1410.png")])

test_loader = DataLoader(images, batch_size=batch_size, shuffle=False)
predictions = predict(model, test_loader, device='cpu')

print(f"Predicted age for the person on the picture: {decode[predictions[0]]}")

Test:   0%|          | 0/1 [00:00<?, ?it/s]

Predicted age for the person on the picture: 50-60
